## 12. Hoeffding Tree and VFDT

### We have used bank dataset to test our implementation (https://www.kaggle.com/janiobachmann/bank-marketing-dataset)

In [38]:
import numpy as np
import pandas as pd
from itertools import chain, combinations
from random import randrange as rr
import math as m
from sklearn.model_selection import train_test_split

dataset = pd.read_csv('bank.csv')
temp = np.array(dataset)

# splitting the dataset
x_train,x_test,y_train,y_test = train_test_split(temp[:,:-1],temp[:,-1],test_size=0.1,random_state=42)

# list of feature names and it's column number
feature_names = dataset.columns[:-1]
feature_index = {}

for i in range(len(feature_names)):
    feature_index[feature_names[i]] = i 

# minimum samples required to check for splitting condition
n_min   = 100
           
print("Features and target: ",feature_names)
print("Number of training samples in stream: ", len(x_train))
print("Number of testing samples: ", len(x_test))

# node class stores the required data and sufficient statistics of each node
class Node:
    def __init__(self, height = 0, index=0):
        self.feature_val = 'Leaf'
        self.split_feature = 'Leaf'
        self.childs = []
        self.height = height
        self.uni_class = set()
        self.index = index
        self.nl = 0
        self.used_features = set()
        self.x_train = []
        self.y_train = []
        self.parent = 0
        
    def __str__(self):
        return "Node-> "+str(self.index)+", Parent id->"+str(self.parent)+", Split Feature: "+ str(self.split_feature)+",  height: "+str(self.height)
            
root = Node(height=1,index=1)

# print(root)
# function to calculate information of a target variable
def info(target_list):
    
    # to avoid runtime error in case of a splitinfo 
    # of an attribute with all same values
    i=0.000000001
    uni_vals = set(target_list)
    
    for ele in uni_vals:
        p = target_list.count(ele)/len(target_list)
        i -= p*m.log2(p)
    return i

# function to find information of a feature column
def info_feature(x,y):
    
    info_f = 0
    uni_features = set(x)

    temp = {}
    
    for ele in uni_features:
        temp[ele] = []

    for i in range(len(x)):
        temp[x[i]].append(y[i])
        
    for ele in uni_features:
        d = len(temp[ele])/len(x)
        info_f -= d*info(temp[ele])
        
    return info_f

n_count = 0
index=2
uni_class = set()
n2=0

# parameters of Hoeffding Tree
delta = 0.0000001
e = 0
t = 0.1

# function to generate stream data from training set
def StreamGenerator():

# the function iterates through the training set and yields one entry each time
    for i in range(len(x_train)):

        yield(x_train[i],y_train[i])

stream = StreamGenerator()

# start reading the stream
for x,y in stream:
    
    n_count+=1
    node = root
    
    # find the suitable node for current data point
    while (node.split_feature!='Leaf'):
        ind = feature_index[node.split_feature]
        
        flag=1
        for c in node.childs:
            i+=1
            if c.feature_val == x[ind]:
                node = c
                flag=0
            # print(c.feature_val,x[ind])
        # print(node)
        if flag==1:
            nn = Node(height=node.height+1, index = index)
            nn.feature_val=x[ind]
            nn.parent = node.index
            nn.used_features = node.used_features.copy()
            index+=1
            node.childs.append(nn)
            
    # update the node data 
    node.nl+=1
    node.uni_class.add(y)
    node.x_train.append(x)
    node.y_train.append(y)
    
    # condition one of the algorithm
    if (node.nl % n_min == 0) and len(node.uni_class)>0:
        
        r1=0
        r2=0
        f1='None'
        f2='None'
        info_y = info(node.y_train)
        f_col = []
        
        # find a feature which is not yet used in the current branch
        # find the max 2 gain ratio among those features
        for f_name in feature_names:
            if f_name not in node.used_features:
                
                ind = feature_index[f_name]
#                 print(ind)
                feature_col = [ele[ind] for ele in node.x_train]
            
                gain = info_y - info_feature(feature_col,node.y_train)
                splitinfo = info(feature_col)
             
                gain_ratio = gain/splitinfo
                
                if gain_ratio > r1:
                    r1,r2 = gain_ratio,r1
                    f1,f2 = f_name,f1
                    f_col = feature_col
                elif gain_ratio > r2:
                    r2 = gain_ratio
                    f2 = f_name
                
        R = m.log2(len(node.uni_class))

        # calculate epsilon
        e = m.sqrt(R*R*m.log(1/delta)/(2*node.nl))

        # 2nd condition of the algorithm
        if f1 != 'None' and ((r1-r2)>e or e<t):
            
            # split the node into childs with unique values of the split features
            
            node.split_feature = f1

            node.used_features.add(f1)
            
            feature_vals = set(f_col)
            
            for ele in feature_vals:
                nn = Node(height=node.height+1, index = index)
                nn.feature_val=ele
                nn.parent = node.index
                nn.used_features = node.used_features.copy()
                index+=1
                node.childs.append(nn)
                
            sep_string = ("   ")*node.height
            print(sep_string,"At sample",n_count,"with epsilon",e)
            print(sep_string,node,"splitted")
            print(sep_string,"into",len(node.childs),"childs\n")
            n2+=node.nl

# predict function which gives accuracy 
def predict(x,y):

    accuracy = 0

    for i in range(len(x)):
        node = root
        
        # find the suitable node for current data point
        while node.split_feature!='Leaf':
            ind = feature_index[node.split_feature]
            flag = 1
            for c in node.childs:
                if c.feature_val == x[i][ind]:
                    node = c
                    flag=0
                break
            if flag==1:
                break
        
        score = node.y_train.count(y[i])/len(node.y_train)
        accuracy += score
        
    accuracy = 100*accuracy/len(y)
    print("The accuracy on the test set is:",accuracy)
    
print("As the training dataset has approx 4000 entries, it is not possible to split \nthe features having high number of unique values, \nas epsilon requires around 10000 entries to reach acceptable value (around 0.03)\n")
predict(x_test,y_test)

Features and target:  Index(['job', 'marital', 'education', 'housing', 'loan', 'contact', 'campaign',
       'poutcome'],
      dtype='object')
Number of training samples in stream:  4068
Number of testing samples:  453
    At sample 100 with epsilon 0.2838846213777555
    Node-> 1, Parent id->0, Split Feature: loan,  height: 1 splitted
    into 2 childs

       At sample 215 with epsilon 0.2838846213777555
       Node-> 2, Parent id->1, Split Feature: poutcome,  height: 2 splitted
       into 4 childs

       At sample 803 with epsilon 0.2838846213777555
       Node-> 3, Parent id->1, Split Feature: poutcome,  height: 2 splitted
       into 4 childs

          At sample 807 with epsilon 0.14194231068887775
          Node-> 6, Parent id->2, Split Feature: housing,  height: 3 splitted
          into 2 childs

          At sample 1284 with epsilon 0.2838846213777555
          Node-> 7, Parent id->2, Split Feature: contact,  height: 3 splitted
          into 2 childs

             At samp